In [1]:
#Importing our Packages
#If anything doesn't work, use !pip install package_name

import pandas as pd
import numpy as np
import matplotlib
import os
import datasets
from PIL import Image

In [2]:
#Data Cleaning?

input_directory = '/Users/Arya/Documents/Data Science/Spring Hackathon/USDA_Hackathon/data/Path2/Path2-Model Training/Path2 Training Images'
output_directory = '/Users/Arya/Documents/Data Science/Spring Hackathon/USDA_Hackathon/data/Path2/Path2-Model Training/Training-PNG'

os.makedirs(output_directory, exist_ok=True)

# List all files in the directory
all_files = os.listdir(input_directory)
all_files

# Filter for .tif files
tif_files = [file for file in all_files if file.endswith('.tif')]

for tif_file in tif_files:
    # Construct the full file path
    full_path = os.path.join(input_directory, tif_file)
    
    with Image.open(full_path) as img:

        base_name = os.path.splitext(tif_file)[0]
        output_path_png = os.path.join(output_directory, f'{base_name}.png')
        img.save(output_path_png, 'PNG')

In [3]:
#Loading Data from Segments.ai
#!pip install --upgrade segments-ai
from segments import SegmentsClient
from segments.utils import export_dataset

api_key = "7b8fe6e8c2ee6ebfc9a2df2a4397ab7d80375620" #API KEY
client = SegmentsClient(api_key)

datasets = client.get_datasets()
print(datasets)

#dataset_name = "ribeye_and_fat_label"

dataset_identifier = 'aamarnath1/USDA_Hackathon-clone'
name = 'v1.0'
release = client.get_release(dataset_identifier, name)
#print(release)

[Dataset(name='USDA_Hackathon-clone', full_name='aamarnath1/USDA_Hackathon-clone', cloned_from='aamarnath1/USDA_Hackathon', description='', category='agriculture', public=False, owner=Owner(username='aamarnath1', created_at='2024-04-06T00:44:52.708856Z', email=None), created_at='2024-04-06T04:51:52.297181Z', enable_ratings=False, enable_skip_labeling=True, enable_skip_reviewing=False, enable_save_button=True, enable_label_status_verified=False, enable_same_dimensions_track_constraint=False, enable_interpolation=True, use_timestamps_for_interpolation=True, task_type='segmentation-bitmap', label_stats=LabelStats(REVIEWED=38, REVIEWING_IN_PROGRESS=None, LABELED=None, LABELING_IN_PROGRESS=None, REJECTED=2, PRELABELED=None, SKIPPED=None, VERIFIED=None, UNLABELED=None, TOTAL=40), labeling_inactivity_timeout_seconds=300, samples_count=40, collaborators_count=0, task_attributes=TaskAttributes(format_version='0.1', categories=[TaskAttributeCategory(name='Ribeye', id=1, color=(0, 113, 188), has_

In [4]:
hfdataset = release2dataset(release, download_images=True)

NameError: name 'release2dataset' is not defined

In [ ]:
#Data Pre-Processing - how can we change the image to seperate the fat + ribeye area?

In [ ]:
#Model Training

In [ ]:
#Model Evaluation

In [ ]:
#Objective -> Classify the beef images 